In [8]:
from openai import AzureOpenAI 

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
import os
client = AzureOpenAI(api_key=os.getenv("AZURE_OPENAI_KEY"), 
                     api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
                     azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
                     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"))




print("************")

api_key=os.getenv("AZURE_OPENAI_KEY")
print(f"************api_key= {api_key}")

completion = client.chat.completions.create(
    
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

TypeError: AzureOpenAI.__init__() got an unexpected keyword argument 'deployment'